In [1]:
import pandas as pd
from datetime import datetime
import os

In [2]:
CURRENT_FILE = "current_snapshot.xlsx"
MASTER_FILE = "master_atm_log.xlsx"

In [3]:
df = pd.read_excel(CURRENT_FILE)

FileNotFoundError: [Errno 2] No such file or directory: 'current_snapshot.xlsx'

In [ ]:
rerequired_cols = ['ATM_ID', 'CAS100', 'CAS200', 'CAS500']
missing = set(required_cols) - set(df.columns)
if missing:
    raise ValueError(f"Missing columns: {missing}")

In [ ]:

df['total_amount'] = df['CAS100'] + df['CAS200'] + df['CAS500']

In [ ]:
df['timestamp'] = datetime.now()


In [ ]:
if os.path.exists(MASTER_FILE):
    master_df = pd.read_excel(MASTER_FILE)
    master_df = pd.concat([master_df, df], ignore_index=True)
else:
    master_df = df.copy()

master_df.to_excel(MASTER_FILE, index=False)

In [ ]:
def check_atm_status(master_df, window=3):
    result = []

    for atm_id, group in master_df.groupby('ATM_ID'):
        recent = group.sort_values('timestamp').tail(window)

        if recent['total_amount'].nunique() == 1 and len(recent) == window:
            status = "DOWN"
        else:
            status = "OK"

        result.append({
            "ATM_ID": atm_id,
            "status": status,
            "last_amount": recent['total_amount'].iloc[-1],
            "last_updated": recent['timestamp'].iloc[-1]
        })

    return pd.DataFrame(result)

status_df = check_atm_status(master_df)


In [ ]:
from openpyxl import load_workbook
from openpyxl.styles import PatternFill

status_df.to_excel("atm_status.xlsx", index=False)

wb = load_workbook("atm_status.xlsx")
ws = wb.active

red_fill = PatternFill(start_color="FF9999", end_color="FF9999", fill_type="solid")

for row in range(2, ws.max_row + 1):
    if ws[f"B{row}"].value == "DOWN":
        for col in range(1, ws.max_column + 1):
            ws.cell(row=row, column=col).fill = red_fill

wb.save("atm_status.xlsx")
